In [1]:
from gensim.scripts.glove2word2vec import glove2word2vec
import pandas as pd
import numpy as np
import gzip
import json
import re
import os
import sys
import prince
import category_encoders as ce   # version 1.2.8
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from fuzzywuzzy import fuzz
import seaborn as sns

sns.set()


%matplotlib inline

Getting user preferences from the data, which genres does the user prefer and average stars.

In [4]:
ratings = pd.read_csv('goodreads_interactions.csv')

In [5]:
bookmap = pd.read_csv('book_id_map.csv')

In [6]:
books = pd.read_csv('cleaned_3tags-books_22-09.csv')

In [7]:
books.head()

,tag1,tag2,tag3,book_id,popular_shelves
0,historical-fiction,mystery,historical-mystery,287141,"['historical-fiction', 'mystery', 'historical-..."
1,short-stories,fiction,french,6066812,"['short-stories', 'fiction', 'french']"
2,fiction,christian-fiction,christian,287149,"['fiction', 'christian-fiction', 'christian']"
3,paranormal,shifters,paranormal-romance,6066814,"['paranormal', 'shifters', 'paranormal-romance']"
4,graphic-novels,ff-re-2011-till-2015,linseyschussan,89375,"['graphic-novels', 'ff-re-2011-till-2015', 'li..."


In [8]:
bookmap.head()

,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361


In [9]:
ratings.head()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [10]:
# replacing book_id with goodread book_id
book_dict = bookmap.set_index('book_id_csv').T.to_dict('list')
ratings['book_id_gr'] = ratings['book_id'].map(book_dict)
ratings['book_id_gr'] = ratings['book_id_gr'].str.get(0)
ratings_clean = ratings.drop(['book_id' ] , axis='columns')

In [11]:
ratings_clean.head()

,user_id,is_read,rating,is_reviewed,book_id_gr
0,0,1,5,0,12
1,0,1,5,1,21
2,0,1,5,0,30
3,0,1,5,0,45
4,0,1,5,0,1


Now that we have a table with user-based info, let's group them by user_id

In [12]:
#ratings_clean.to_csv('ratings_clean_all.csv', index=False)

In [2]:
ratings_clean = pd.read_csv('ratings_clean_all.csv', dtype={'book_id_gr':str})

In [3]:
ratings_clean.head()

,user_id,is_read,rating,is_reviewed,book_id_gr
0,0,1,5,0,12
1,0,1,5,1,21
2,0,1,5,0,30
3,0,1,5,0,45
4,0,1,5,0,1


In [6]:
ratings_clean['book_id_gr'].iloc[0]

'12'

In [28]:
sum_ratings_user = ratings_clean.groupby('user_id')['rating'].count()

In [19]:
ave_ratings_user = ratings_clean.groupby('user_id')['rating'].mean()

In [22]:
sum_ratings_user.to_csv('sum_ratings_user.csv', index=False)
ave_ratings_user.to_csv('ave_ratings_user.csv', index=False)

/Users/yasminechebaro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/Users/yasminechebaro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [7]:
ratings_read = ratings_clean[ratings_clean['is_read'] == 1]
ratings_reviewed = ratings_clean[ratings_clean['is_reviewed'] == 1]

In [ ]:
ratings_read.to_csv('ratings_read.csv')
ratings_reviewed.to_csv('ratings_reviewed.csv')

In [ ]:
ave_ratings_user_read = ratings_read.groupby('user_id')['rating'].mean()
ave_ratings_user_read.to_csv('ave_ratings_user_read_books.csv')

In [32]:
ratings_reviewed.head()

,user_id,is_read,rating,is_reviewed,book_id_gr
1,0,1,5,1,21
9,0,1,5,1,2
10,0,1,5,1,6
14,0,1,5,1,1022863
18,0,1,2,1,828


In [33]:
sum_reviewed_user = ratings_reviewed.groupby('user_id')['is_reviewed'].count()

In [37]:
sum_read_user = ratings_read.groupby('user_id')['is_read'].count()

So far per user, the information collected is the number of reviews, the number of books read, and the number of entries per user.

In [39]:
sum_read_user.to_csv('sum_read_user.csv')
sum_reviewed_user.to_csv('sum_reviewed_user.csv')

/Users/yasminechebaro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/Users/yasminechebaro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [40]:
books_user = ratings_read.groupby('user_id')['book_id_gr']

In [43]:
#books_user.head()

In [42]:
ratings_read.head()

,user_id,is_read,rating,is_reviewed,book_id_gr
0,0,1,5,0,12
1,0,1,5,1,21
2,0,1,5,0,30
3,0,1,5,0,45
4,0,1,5,0,1


In [ ]:
ratings_read2 = ratings_read.copy()
ratings_read2['book_id_gr'] = ratings_read2['book_id_gr'].astype(str)

In [44]:
user_books = ratings_read2.groupby('user_id')['book_id_gr'].apply(','.join).reset_index()

TypeError: sequence item 0: expected str instance, int found